<a href="https://colab.research.google.com/github/alexlinapp/proofLLM/blob/main/finetuning_reasoning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Authentication. Make sure you have authentication token as well as access to the gated repo

from huggingface_hub import login

login()

import torch
import transformers

# model_id = "meta-llama/Meta-Llama-3-8B"
# pipeline = transformers.pipeline("text-generation", model=model_id, model_kwargs={"torch_dtype" : torch.bfloat16}, device_map="auto")
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("gpt2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
#print(model.lm_head)
x = model(torch.tensor([[1, 2, 3]]))

tensor([[[-32.9011, -31.2024, -34.6622,  ..., -39.4867, -39.8731, -32.2387],
         [-55.5208, -53.4285, -56.4767,  ..., -68.1539, -66.7709, -58.6006],
         [-61.7969, -60.5386, -59.5503,  ..., -75.3206, -72.7731, -65.5706]]],
       grad_fn=<UnsafeViewBackward0>)

In [ ]:
# config = model.config
# output = model(torch.tensor([[1, 2, 3]]))
# print(output.logits)
# for key, value in model.named_parameters():
#   print(key)
for params in model.parameters():
  params.requires_grad = False
for params in model.lm_head.parameters():
  params.requires_grad = True

In [ ]:
from datasets import load_dataset

# testing data
dataset = load_dataset("cais/mmlu", "abstract_algebra")
print(dataset.keys())
print(type(dataset["test"]))
print(len(dataset["test"]))



README.md: 0.00B [00:00, ?B/s]

dataset_infos.json: 0.00B [00:00, ?B/s]

abstract_algebra/test-00000-of-00001.par(…):   0%|          | 0.00/9.96k [00:00<?, ?B/s]

abstract_algebra/validation-00000-of-000(…):   0%|          | 0.00/3.73k [00:00<?, ?B/s]

abstract_algebra/dev-00000-of-00001.parq(…):   0%|          | 0.00/3.45k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/5 [00:00<?, ? examples/s]

dict_keys(['test', 'validation', 'dev'])
<class 'datasets.arrow_dataset.Dataset'>
100


In [ ]:
dataset = load_dataset("entfane/professor-mathematics")

# each element in the dataset is of type dict
print(dataset["train"][64202])



README.md:   0%|          | 0.00/335 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/61.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/64203 [00:00<?, ? examples/s]

{'question': 'At 30, Anika is 4/3 the age of Maddie. What would be their average age in 15 years?', 'answer': "If Anika is 30 now, in 15 years, she'll be 30+15=<<30+15=45>>45 years old.\nAt 30, Anika is 4/3 the age of Maddie, meaning Maddie is 4/3*30=<<4/3*30=40>>40 years.\nIn 15 years, Maddie will be 40+15=<<40+15=55>>55 years old.\nTheir total age in 15 years will be 55+45=<<55+45=100>>100\nTheir average age in 15 years will be 100/2=<<100/2=50>>50\n#### 50"}


In [ ]:
print(((dataset["train"][:2])))

{'question': ['Let \\( a, b, c \\) be positive real numbers. Prove that\n\n$$\n\\frac{1}{a(1+b)}+\\frac{1}{b(1+c)}+\\frac{1}{c(1+a)} \\geq \\frac{3}{1+abc},\n$$\n\nand that equality occurs if and only if \\( a = b = c = 1 \\).', 'Given real numbers \\( a, b, c \\) and a positive number \\( \\lambda \\) such that the polynomial \\( f(x) = x^3 + a x^2 + b x + c \\) has three real roots \\( x_1, x_2, x_3 \\), and the conditions \\( x_2 - x_1 = \\lambda \\) and \\( x_3 > \\frac{1}{2}(x_1 + x_2) \\) are satisfied, find the maximum value of \\( \\frac{2 a^3 + 27 c - 9 a b}{\\lambda^3} \\).'], 'answer': ["1. Consider the given inequality:\n\n\\[\n\\frac{1}{a(1+b)}+ \\frac{1}{b(1+c)}+ \\frac{1}{c(1+a)} \\geq \\frac{3}{1 + abc}\n\\]\n\nTo simplify, we add \\( \\frac{3}{1 + abc} \\) to both sides. The new inequality becomes:\n\n\\[\n\\frac{1}{a(1+b)} + \\frac{1}{b(1+c)} + \\frac{1}{c(1+a)} + \\frac{3}{1 + abc} \\geq \\frac{6}{1 + abc}\n\\]\n\n2. Let's analyze each term with an added \\( \\frac{1

In [ ]:
from transformers import AutoTokenizer


BASE_CONFIG = {
    "context_length": 512
}
tokenizer = AutoTokenizer.from_pretrained("gpt2")

def format_input(entry):
  instruction_text = (
      f"Below is a question that would like a logical answer. Generate a response that provides a step by step solution."
      f"\n\n### Instruction: {entry['question']}"
      f"\n\n### Answer: {entry['answer']}"
  )
  return instruction_text




def pre_process(entry):
  input_text = format_input(entry)
  return tokenizer(input_text, return_tensors="pt")



tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
# train_data, val_data, test_data = split_data(dataset)
# train_dataset = dataset["train"][:2]
# type(train_data)
# print(dataset["train"])
print(dataset)
split = dataset["train"].train_test_split(test_size=0.1)
smalldataset = split["train"].select(range(1000))
small_train = smalldataset.map(pre_process)
small_val = split["test"].select(range(1000)).map(pre_process)

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 64203
    })
})


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1194 > 1024). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
small_train
small_train.set_format(type="torch", columns=["input_ids"])
small_train

Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask'],
    num_rows: 1000
})

In [ ]:
type(small_train[0])
len(small_train)

1000

In [ ]:
from torch.utils.data import DataLoader
import torch
from functools import partial
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
customized_collate_fn = partial(
 custom_collate_fn,
 device=device,
 allowed_max_length=1024
)
#torch.manual_seed(123)
train_loader = DataLoader(small_train, batch_size=1, shuffle=True, collate_fn=customized_collate_fn, drop_last=True)
val_loader = DataLoader(small_val, batch_size=6, shuffle=False, collate_fn=customized_collate_fn, drop_last=False)
next(iter(train_loader))


(tensor([[21106,   318,   257,  ...,  3524,   276, 31478]]),
 tensor([[  318,   257,  1808,  ...,   276, 31478, 50256]]))

In [ ]:
input_text = token_ids_to_text(inputs, tokenizer)
print(generated_text[len(input_text):])

 \frac{3k


In [ ]:
model.to(device)
torch.manual_seed(123)
inputs, targets = next(iter(train_loader))

token_ids_to_text(inputs, tokenizer)
output = generate(model=model, idx=inputs, context_size=1024, max_new_tokens=5, eos_id=50256)
print("This is output: ", output)
generated_text = token_ids_to_text(output, tokenizer)
print("\n\n==============\n\n")
print(generated_text)

This is output:  tensor([[21106,   318,   257,  ...,    90,    18,    74]])




Below is a question that would like a logical answer. Generate a response that provides a step by step solution.

### Instruction: Let the sides of a triangle be \(a, b, c\) and the semi-perimeter be \(p\). Prove that \(\sqrt{p-a} + \sqrt{p-b} + \sqrt{p-c} \leq \sqrt{3p}\).

### Answer: ### Problem:
Let \( a, b, c \) be the lengths of the sides of a triangle, and let \( p \) denote the semiperimeter of the triangle. Prove the inequality:
\[
\sqrt{p-a} + \sqrt{p-b} + \sqrt{p-c} \leq \sqrt{3p}
\]

1. **Choose a Parameter:**
   Let \( k \) be an arbitrary positive parameter. We start by expressing each square root term with \( k \):
   \[
   \sqrt{p-a} + \sqrt{p-b} + \sqrt{p-c} = \frac{\sqrt{k(p-a)} + \sqrt{k(p-b)} + \sqrt{k(p-c)}}{\sqrt{k}}
   \]

2. **Use the Arithmetic Mean Inequality (AM-GM Inequality):**
   By the Arithmetic Mean-Geometric Mean Inequality (AM-GM), for any non-negative numbers \( x_1, x_2,

In [ ]:
input_text = format_input(dataset["train"][0])
token_ids = generate(
 model=model,
 idx=text_to_token_ids(input_text, tokenizer),
 max_new_tokens=35,
 context_size=BASE_CONFIG["context_length"],
 eos_id=50256,
)
generated_text = token_ids_to_text(token_ids, tokenizer)
response_text = generated_text[len(input_text):].strip()
print(response_text)

KeyboardInterrupt: 

In [ ]:
import time



start_time = time.time()
torch.manual_seed(123)
optimizer = torch.optim.AdamW(
 model.parameters(), lr=0.00005, weight_decay=0.1
)
num_epochs = 2
train_losses, val_losses, tokens_seen = train_model_simple(
 model, train_loader, val_loader, optimizer, device,
 num_epochs=num_epochs, eval_freq=5, eval_iter=5,
 start_context=format_input(val_data[0]), tokenizer=tokenizer
)
end_time = time.time()
execution_time_minutes = (end_time - start_time) / 60
print(f"Training completed in {execution_time_minutes:.2f} minutes.")

In [ ]:
print(format_input(dataset["train"][0]))

Below is a question that would like a logical answer. Generate a response that provides a step by step solution.

### Instruction: Let \( a, b, c \) be positive real numbers. Prove that

$$
\frac{1}{a(1+b)}+\frac{1}{b(1+c)}+\frac{1}{c(1+a)} \geq \frac{3}{1+abc},
$$

and that equality occurs if and only if \( a = b = c = 1 \).

### Answer: 1. Consider the given inequality:

\[
\frac{1}{a(1+b)}+ \frac{1}{b(1+c)}+ \frac{1}{c(1+a)} \geq \frac{3}{1 + abc}
\]

To simplify, we add \( \frac{3}{1 + abc} \) to both sides. The new inequality becomes:

\[
\frac{1}{a(1+b)} + \frac{1}{b(1+c)} + \frac{1}{c(1+a)} + \frac{3}{1 + abc} \geq \frac{6}{1 + abc}
\]

2. Let's analyze each term with an added \( \frac{1}{1 + abc} \):

\[
\frac{1}{a(1+b)} + \frac{1}{1 + abc}, \quad \frac{1}{b(1+c)} + \frac{1}{1 + abc}, \quad \frac{1}{c(1+a)} + \frac{1}{1 + abc}
\]

We can rewrite them as follows:

\[
\begin{aligned}
\frac{1}{a(1+b)} + \frac{1}{1 + abc} &= \frac{1}{1 + abc} \left( \frac{1 + abc}{a(1+b)} + 1 \righ

In [ ]:
def custom_collate_fn(batch, pad_token_id=50256,
                      ignore_index=-100,
                      allowed_max_length=None,
                      device="cpu"):
  batch_max_length = max(item['input_ids'].shape[1] + 1 for item in batch)
  if allowed_max_length is not None:
    batch_max_length = min(batch_max_length, allowed_max_length+1)


  inputs_lst, targets_lst = [], []
  for item in batch:
    new_item = item['input_ids'][0].clone()
    pad_length = batch_max_length - new_item.shape[0]
    padding = torch.full((max(0, pad_length),), pad_token_id)
    padded = torch.cat([new_item[:batch_max_length], padding], dim=0)
    inputs = padded[:-1].clone()
    targets = padded[1:].clone()
    targets[targets.shape[0]-1] = pad_token_id
    mask = targets == pad_token_id
    indices = torch.nonzero(mask).squeeze()
    if indices.numel() > 1:
      targets[indices[1:]] = ignore_index


    inputs_lst.append(inputs)
    targets_lst.append(targets)
  inputs_tensor = torch.stack(inputs_lst, dim=0).to(device)
  targets_tensor = torch.stack(targets_lst, dim=0).to(device)
  return inputs_tensor, targets_tensor


In [ ]:
def calc_loss_batch(input_batch, target_batch, model, device):
    input_batch, target_batch = input_batch.to(device), target_batch.to(device)
    logits = model(input_batch)
    loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), target_batch.flatten())
    return loss


def calc_loss_loader(data_loader, model, device, num_batches=None):
    total_loss = 0.
    if len(data_loader) == 0:
        return float("nan")
    elif num_batches is None:
        num_batches = len(data_loader)
    else:
        # Reduce the number of batches to match the total number of batches in the data loader
        # if num_batches exceeds the number of batches in the data loader
        num_batches = min(num_batches, len(data_loader))
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            total_loss += loss.item()
        else:
            break
    return total_loss / num_batches



def generate_text_simple(model, idx, max_new_tokens, context_size):
    # idx is (B, T) array of indices in the current context
    for _ in range(max_new_tokens):

        # Crop current context if it exceeds the supported context size
        # E.g., if LLM supports only 5 tokens, and the context size is 10
        # then only the last 5 tokens are used as context
        idx_cond = idx[:, -context_size:]

        # Get the predictions
        with torch.no_grad():
            logits = model(idx_cond)

        # Focus only on the last time step
        # (batch, n_token, vocab_size) becomes (batch, vocab_size)
        logits = logits[:, -1, :]

        # Get the idx of the vocab entry with the highest logits value
        idx_next = torch.argmax(logits, dim=-1, keepdim=True)  # (batch, 1)

        # Append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1)  # (batch, n_tokens+1)

    return idx


def generate(model, idx, max_new_tokens, context_size, temperature=0.0, top_k=None, eos_id=None):

    # For-loop is the same as before: Get logits, and only focus on last time step
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -context_size:]
        with torch.no_grad():
            output = model(idx_cond)
            if isinstance(output, transformers.utils.ModelOutput):
                logits = output.logits
            else:
                logits = output
        logits = logits[:, -1, :]

        # New: Filter logits with top_k sampling
        if top_k is not None:
            # Keep only top_k values
            top_logits, _ = torch.topk(logits, top_k)
            min_val = top_logits[:, -1]
            logits = torch.where(logits < min_val, torch.tensor(float('-inf')).to(logits.device), logits)

        # New: Apply temperature scaling
        if temperature > 0.0:
            logits = logits / temperature

            # Apply softmax to get probabilities
            probs = torch.softmax(logits, dim=-1)  # (batch_size, context_len)

            # Sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (batch_size, 1)

        # Otherwise same as before: get idx of the vocab entry with the highest logits value
        else:
            idx_next = torch.argmax(logits, dim=-1, keepdim=True)  # (batch_size, 1)

        if idx_next == eos_id:  # Stop generating early if end-of-sequence token is encountered and eos_id is specified
            break

        # Same as before: append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1)  # (batch_size, num_tokens+1)

    return idx


def train_model_simple(model, train_loader, val_loader, optimizer, device, num_epochs,
                       eval_freq, eval_iter, start_context, tokenizer):
    # Initialize lists to track losses and tokens seen
    train_losses, val_losses, track_tokens_seen = [], [], []
    tokens_seen, global_step = 0, -1

    # Main training loop
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode

        for input_batch, target_batch in train_loader:
            optimizer.zero_grad()  # Reset loss gradients from previous batch iteration
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            loss.backward()  # Calculate loss gradients
            optimizer.step()  # Update model weights using loss gradients
            tokens_seen += input_batch.numel()
            global_step += 1

            # Optional evaluation step
            if global_step % eval_freq == 0:
                train_loss, val_loss = evaluate_model(
                    model, train_loader, val_loader, device, eval_iter)
                train_losses.append(train_loss)
                val_losses.append(val_loss)
                track_tokens_seen.append(tokens_seen)
                print(f"Ep {epoch+1} (Step {global_step:06d}): "
                      f"Train loss {train_loss:.3f}, Val loss {val_loss:.3f}")

        # Print a sample text after each epoch
        generate_and_print_sample(
            model, tokenizer, device, start_context
        )

    return train_losses, val_losses, track_tokens_seen


def evaluate_model(model, train_loader, val_loader, device, eval_iter):
    model.eval()
    with torch.no_grad():
        train_loss = calc_loss_loader(train_loader, model, device, num_batches=eval_iter)
        val_loss = calc_loss_loader(val_loader, model, device, num_batches=eval_iter)
    model.train()
    return train_loss, val_loss


def generate_and_print_sample(model, tokenizer, device, start_context):
    model.eval()
    context_size = model.pos_emb.weight.shape[0]
    encoded = text_to_token_ids(start_context, tokenizer).to(device)
    with torch.no_grad():
        token_ids = generate_text_simple(
            model=model, idx=encoded,
            max_new_tokens=50, context_size=context_size
        )
        decoded_text = token_ids_to_text(token_ids, tokenizer)
        print(decoded_text.replace("\n", " "))  # Compact print format
    model.train()


def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text)
    encoded_tensor = torch.tensor(encoded).unsqueeze(0)  # add batch dimension
    return encoded_tensor


def token_ids_to_text(token_ids, tokenizer):
    flat = token_ids.squeeze(0)  # remove batch dimension
    return tokenizer.decode(flat.tolist())
